# qiskit_alt

This python package provides a thin wrapper around some features of Qiskit that have been (re-)implemented in Julia and provides a Python interface. The input and output of the Python interface are the same as the input and output to Python qiskit. At present, we have prepared two high level demonstrations

* Performing the Jordan-Wigner transform from a Fermionic operator to a Pauli operator.

* Computing the Fermionic operator from integrals computed by `pyscf`.

In both cases, we will see that the `qiskit_alt` implementation is much more performant.

We have also prepared some lower-level demonstrations of performance gains

* Converting an operator from the computational basis to the Pauli basis.

The Python package has been installed in a virtual environment created with `python -m venv ./env`. The Julia packages have been installed in a local environment in the standard way, via a spec in `Project.toml` file.

When we import the package `qiskit_alt`, the Julia environment is also activated.

In [1]:
import qiskit_alt  # The message below is from Julia

  Activating project at `~/myrepos/quantum_repos/qiskit_alt`


We assume that no one is familiar with Julia, much less with `pyjulia`, the package we use to call Julia from Python. So, we inject a bit of tutorial.

The default `Module` in Julia `Main` is available. You can think of it as a namespace. And, as always, objects from the `Module` `Base` have been imported into `Main`.

As an example of how `pyjulia` works, we create an `Array` of `Float64` zeros on the Julia side, and they are automatically copied to a numpy array when returned to Python.

In [2]:
qiskit_alt.Main.zeros(10)

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall ~/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

There are several ways to call Julia from Python and vice versa, and to specifiy features such as the copying vs. sharing semantics. We won't go into any of this in this demo.

## Electronic structure

Part of a workflow for, say, VQE involves using qiskit-nature to do the following:
* Get a description of a model Hamiltonian from the package `pyscf` by passing it a description of the geometry of a molecule.
* Convert that description of a Hamiltonian to a qiskit-native Fermionic operator.
* Convert the Fermionic operator to a qubit operator expressed in the Pauli basis.

The last step above may be done in several ways, one of which is known as the Jordan-Wigner transform. It is this step that we will benchmark here.

### qiskit-nature

First, we see how this is done in qiskit-nature. We need to specify the geometry of the molecule and the
[basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)). We choose `sto3g`, one of the smallest, simplest, basis sets.

In [3]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver

# Specify the geometry of the H_2 molecule
geometry = [['H', [0., 0., 0.]],
            ['H', [0., 0., 0.735]]]

basis = 'sto3g'

Then, we compute the fermionic Hamiltonian like this.

In [4]:
molecule = Molecule(geometry=geometry,
                     charge=0, multiplicity=1)
driver = ElectronicStructureMoleculeDriver(molecule, basis=basis, driver_type=ElectronicStructureDriverType.PYSCF)

from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

es_problem = ElectronicStructureProblem(driver)
second_q_op = es_problem.second_q_ops()

fermionic_hamiltonian = second_q_op[0]

The Jordan-Wigner transform is performed like this.

In [5]:
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
nature_qubit_op = qubit_converter.convert(fermionic_hamiltonian)
nature_qubit_op.primitive

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.81054798+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### qiskit_alt

The only high-level code in `qiskit_alt` was written to support this demo. So doing the JW-transform is less verbose.

In [6]:
fermi_op = qiskit_alt.fermionic_hamiltonian(geometry, basis)
pauli_op = qiskit_alt.qubit_hamiltonian(fermi_op)
pauli_op.simplify() # The Julia Pauli operators use a different sorting convention; we sort again for comparison. 

SparsePauliOp(['IIII', 'ZIII', 'IZII', 'ZZII', 'IIZI', 'ZIZI', 'IZZI', 'IIIZ', 'ZIIZ', 'IZIZ', 'IIZZ', 'XXXX', 'YYXX', 'XXYY', 'YYYY'],
              coeffs=[-0.09057899+0.j, -0.22575349+0.j,  0.17218393+0.j,  0.12091263+0.j,
 -0.22575349+0.j,  0.17464343+0.j,  0.16614543+0.j,  0.17218393+0.j,
  0.16614543+0.j,  0.16892754+0.j,  0.12091263+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j])

### Benchmarking

Computing the Hamiltonian for a larger molecule or a larger basis set takes more time and produces a Hamiltonian with more factors and terms. Here we compare the performance of `qiskit_alt` and `qiskit-nature` on combinations of $\text{H}_2$ and $\text{H}_2\text{O}$ molecules for several basis sets.

First we benchmark qiskit-nature, which records the times in `nature_times`.

In [7]:
%run ../bench/jordan_wigner_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 8.12 ms
geometry=h2_geometry, basis='631g' 108.14 ms
geometry=h2_geometry, basis='631++g' 563.58 ms
geometry=h2o_geometry, basis='sto3g' 665.99 ms
geometry=h2o_geometry, basis='631g' 19161.12 ms


[8.117510983720422,
 108.13769628293812,
 563.5829328559339,
 665.9909158013761,
 19161.11534414813]

Next we benchmark qiskit_alt, which records the times in `alt_times`.

In [8]:
%run ../bench/jordan_wigner_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 0.81 ms
geometry=h2_geometry, basis='631g' 5.31 ms
geometry=h2_geometry, basis='631++g' 27.45 ms
geometry=h2o_geometry, basis='sto3g' 25.60 ms
geometry=h2o_geometry, basis='631g' 601.39 ms


[0.8140197023749352,
 5.314521677792072,
 27.44855098426342,
 25.59504983946681,
 601.3947255909443]

We compare the relative performance.

In [9]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[9.972130846510542,
 20.347587767835417,
 20.53233823450435,
 26.020301580911077,
 31.861129685365924]

We see that
* qiskit_alt is at least ten times faster
* The relative performance increases as the problem in some sense gets larger.

In fact, another problem, not shown here, finishes in 18s with qiskit_alt and in 5730s in qiskit-nature.
In this case, `qiskit_alt` is 320 times faster than `qiskit-nature`. I don't have an idea about the origin of this scaling.

### Computing the Fermonic operator

Computing the Fermionic operator from the output of `pyscf` is also much more efficient in `qiskit_alt`.

We benchmark qiskit-nature computing the fermionic Hamiltonian

In [10]:
%run ../bench/fermionic_nature_time.py
nature_times

geometry=h2_geometry, basis='sto3g' 58.49 ms
geometry=h2_geometry, basis='631g' 99.97 ms
geometry=h2_geometry, basis='631++g' 293.18 ms
geometry=h2o_geometry, basis='sto3g' 460.98 ms
geometry=h2o_geometry, basis='631g' 27068.14 ms


[58.490909077227116,
 99.97380957938731,
 293.18450139835477,
 460.9842232428491,
 27068.13617516309]

We benchmark qiskit_alt computing the fermionic Hamiltonian.

In [11]:
%run ../bench/fermionic_alt_time.py
alt_times

geometry=h2_geometry, basis='sto3g' 58.58 ms
geometry=h2_geometry, basis='631g' 61.78 ms
geometry=h2_geometry, basis='631++g' 65.26 ms
geometry=h2o_geometry, basis='sto3g' 78.22 ms
geometry=h2o_geometry, basis='631g' 214.97 ms


[58.581056306138635,
 61.77768390625715,
 65.26236040517688,
 78.22452755644917,
 214.97237859293818]

We compare the relative performance.

In [12]:
[t_nature / t_qk_alt for t_nature, t_qk_alt in zip(nature_times, alt_times)]

[0.9984611539191028,
 1.6182835493005827,
 4.492398061886498,
 5.893090538772369,
 125.91448423435864]

We see again that, as the problem size increases, `qiskit_alt` is increasingly more performant.

## Discussion

The Julia implemenation consists of these packages

* [`QuantumOps.jl`](https://github.com/jlapeyre/QuantumOps.jl) implementing Fermionic and Pauli operators and calculations using them.

* [`ElectronicStructure.jl`](https://github.com/jlapeyre/ElectronicStructure.jl) provides an interface to electronic structure packages. Only one is supported, `pyscf`.

* [`QiskitQuantumInfo.jl`](https://github.com/jlapeyre/QiskitQuantumInfo.jl) provides data structures that mirror Python Qiskit data structures. These are used as intermedidate structures for converting from `QuantumOps` and `ElectronicStructure` to Python Qiskit. In the future these might be used directly for calculations.


The Python interface is a Python package `qiskit_alt`. This could contain a mixture of Julia and Python code. Or all the Julia code might be moved to the Julia packages.

### Implementation

In the examples above, the following happens.

* Julia code calls `pyscf` and stores the results in Julia data structures.

* These data are used to construct a Fermionic operator as a data structure defined in `QuantumOps`.

* The Jordan-Wigner transform, implemented in `QuantumOps` is used to compute a Pauli operator.

* The Pauli operator (as a structure in `QuantumOps`) is converted to a Qiskit-like operator defined in `QiskitQuantumInfo.jl`.

* The operator defined in `QiskitQuantumInfo.jl` is sent to Python and converted to numpy arrays, which are then used to construct native Qiskit types. The conversion to numpy arrays is provided by `pyjulia`.

### Complexity, dynamism

* It is worth noting that operators in `QuantumOps` are *not* highly optimized implementations. In fact, much of the code for the two types of operators is shared, they inherit from a parent class. There are other implementations of Pauli operators in Julia that are much more efficient for instance in [`QuantumClifford.jl`](https://github.com/Krastanov/QuantumClifford.jl).

* This performance does not come at the cost of complexity, as with C++. Many people, including myself, find Julia to be more dynamic and expressive than Python. It is easier and faster to put ideas into code. The limitations of the language are less obtrusive.

# More demos

Here are some smaller scale demonstrations.

## Converting a matrix to the Pauli basis

Here we convert a matrix representing an operator in the computational basis to the Pauli basis.
In this case, `qiskit_alt` is much more performant than `qiskit.quantum_info`.
This is how it is done in `QuantumOps`.

In [13]:
from qiskit_alt import QuantumOps, PauliSum_to_SparsePauliOp
import numpy as np
m = np.random.rand(2**3, 2**3) # 3-qubit operator
pauli_sum = QuantumOps.PauliSum(m) # This is a wrapped Julia object
PauliSum_to_SparsePauliOp(pauli_sum) # Convert to qiskit.quantum_info.SparsePauliOp

SparsePauliOp(['III', 'XII', 'YII', 'ZII', 'IXI', 'XXI', 'YXI', 'ZXI', 'IYI', 'XYI', 'YYI', 'ZYI', 'IZI', 'XZI', 'YZI', 'ZZI', 'IIX', 'XIX', 'YIX', 'ZIX', 'IXX', 'XXX', 'YXX', 'ZXX', 'IYX', 'XYX', 'YYX', 'ZYX', 'IZX', 'XZX', 'YZX', 'ZZX', 'IIY', 'XIY', 'YIY', 'ZIY', 'IXY', 'XXY', 'YXY', 'ZXY', 'IYY', 'XYY', 'YYY', 'ZYY', 'IZY', 'XZY', 'YZY', 'ZZY', 'IIZ', 'XIZ', 'YIZ', 'ZIZ', 'IXZ', 'XXZ', 'YXZ', 'ZXZ', 'IYZ', 'XYZ', 'YYZ', 'ZYZ', 'IZZ', 'XZZ', 'YZZ', 'ZZZ'],
              coeffs=[ 3.88381220e-01+0.j        ,  5.32465330e-01+0.j        ,
  0.00000000e+00+0.14908267j, -2.08051975e-01+0.j        ,
  4.90641943e-01+0.j        ,  5.42657962e-01+0.j        ,
  0.00000000e+00-0.03600403j, -1.58045678e-01+0.j        ,
  0.00000000e+00+0.11658117j,  0.00000000e+00-0.02527994j,
  1.18291063e-01+0.j        ,  0.00000000e+00-0.00073637j,
 -1.53158707e-01+0.j        , -9.35718726e-02+0.j        ,
  0.00000000e+00-0.09918959j,  8.08092276e-02+0.j        ,
  5.35804879e-01+0.j        ,  5.76092538e-

Note that the `numpy` matrix was automatically converted to a Julia `Matrix`.

We run benchmarks of conversion of matrices to the Pauli basis.

In [14]:
%run ../bench/from_matrix_quantum_info.py

nqubits=2, 1.16 ms
nqubits=3, 4.72 ms
nqubits=4, 23.24 ms
nqubits=5, 134.58 ms
nqubits=6, 994.52 ms
nqubits=7, 10708.95 ms
nqubits=8, 194281.00 ms


In [15]:
%run ../bench/from_matrix_alt.py

nqubits=2, 0.68 ms
nqubits=3, 0.78 ms
nqubits=4, 11.52 ms
nqubits=5, 28.15 ms
nqubits=6, 27.68 ms
nqubits=7, 113.21 ms
nqubits=8, 603.29 ms


Here are the ratios of the times for `qiskit.quantum_info` to those for `qiskit_alt`.

In [16]:
[t_pyqk / t_qk_alt for t_pyqk, t_qk_alt in zip(pyqk_times, qk_alt_times)]

[1.7070298345532198,
 6.0833434027229805,
 2.016501257638242,
 4.780716334331067,
 35.92772747539492,
 94.59346712443646,
 322.0351650952433]

Again, the performance gain increases with the problem size.

## Creating a `SparsePauliOp` from a list of strings


Here, we create a `SparsePauliOp` from a list of `n` strings, each with `k` single-Pauli factors, and simplify the result.

First, using `qiskit.quantum_info`

In [17]:
%run ../bench/pauli_from_list_qinfo.py

k=10, n=10, 0.00034175594337284567 sec
k=10, n=100, 0.0016513901995494963 sec
k=10, n=1000, 0.013844832358881832 sec
k=10, n=5000, 0.06792515469714999 sec
k=10, n=10000, 0.1374026113189757 sec
k=10, n=100000, 1.390079686883837 sec
k=100, n=10, 0.0003708278527483344 sec
k=100, n=100, 0.001680322433821857 sec
k=100, n=1000, 0.015005420404486358 sec
k=100, n=5000, 0.07447947694454342 sec
k=100, n=10000, 0.15302739711478353 sec
k=100, n=100000, 1.5492611401714385 sec


Now, using `qiskit_alt`

In [18]:
%run ../bench/pauli_from_list_alt.py

k=10, n=10, 0.01730599331203848 sec
k=10, n=100, 0.0007586243329569698 sec
k=10, n=1000, 0.0018087288364768029 sec
k=10, n=5000, 0.008252075244672596 sec
k=10, n=10000, 0.016550637548789382 sec
k=10, n=100000, 0.18883962654508651 sec
k=100, n=10, 0.0007047933991998434 sec
k=100, n=100, 0.0008528033504262567 sec
k=100, n=1000, 0.0027765627484768627 sec
k=100, n=5000, 0.01570416670292616 sec
k=100, n=10000, 0.027043094113469123 sec
k=100, n=100000, 0.3140773644903675 sec


The results were written to lists `quantum_info_times` and `qkalt_times`. We compare the performance:

In [19]:
[x / y for x,y in zip(quantum_info_times, qkalt_times)]

[0.019747837480967458,
 2.176822081507324,
 7.654454376837372,
 8.231281548359773,
 8.301952774564034,
 7.361165197771389,
 0.5261511432560771,
 1.9703515857225251,
 5.404315250111985,
 4.742657051052931,
 5.658649726717716,
 4.932737329496259]

We see that the performance improvement in `qiskit_alt` is significant, but does not increase with the number of terms `n`. Further benchmarks show that the time required to convert the strings from Python to Julia is about three times longer than the time required to construct the operators. 